In [1]:
import os, random, time, pymysql
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import openpyxl
from scipy.stats import mode
import shutil

In [2]:
def _call_db_info(db_name):
    return pymysql.connect(
        host = '10.94.77.9',
        port= 13306,
        user = 'dxbot1',
        password='ensol2020!',
        db = db_name,
        charset = 'utf8')

In [11]:
dir_ = 'CBEV'

In [12]:
conn = _call_db_info('gscm_'+dir_.lower())
curs = conn.cursor()
curs.execute("select * from dt_table;") 
dt_table = pd.DataFrame(curs.fetchall())
conn.commit()
conn.close()
dt_table.columns = ['WW']    

if os.path.isdir('./'+dir_+'/data1/.ipynb_checkpoints'):
    shutil.rmtree('./'+dir_+'/data1/.ipynb_checkpoints')
f_list = os.listdir('./'+dir_+'/data1/')
f_list.sort()
period_file = [i for i, v in enumerate(f_list) if 'PERIODIC' in v.split('_')]

In [13]:
for i in range(0, len(period_file)):
    tmp_ = pd.read_csv('./'+dir_+'/data1/'+f_list[period_file[i]])
    if i == 0 :
        tmp=pd.merge(dt_table, tmp_, how='left', left_on='WW', right_on='YYYYWW')
    else :
        tmp=pd.merge(tmp, tmp_, how='left', left_on='WW', right_on='YYYYWW')
    tmp = tmp.drop(['YYYYWW'], axis=1)
    
if tmp.shape[1] >2:
    tmp1 = tmp.fillna(0)
    tmp1[tmp1 == -999] = 0
    tmp2 = pd.DataFrame({'WW' : tmp1['WW'],
                         'PERIODIC_INVEN_WW': tmp1.iloc[:, 1:].sum(axis=1)})
else:
    tmp1 = tmp.fillna(0)
    tmp1[tmp1 == -999] = 0    
    tmp2 = tmp1.copy()
tmp2 = tmp2[tmp2['PERIODIC_INVEN_WW'].notna()]    

In [14]:
insert_lt_sql = "insert into lt_day values (%s, %s)"#.format(*c)
conn = _call_db_info('gscm_'+dir_.lower())
curs = conn.cursor()
for i in range(0, tmp.shape[0]):
    insert_list = tmp2.loc[i].values.tolist()
    curs.execute(insert_lt_sql, insert_list) 
conn.commit()
conn.close()    

In [7]:
tmp2

,WW,PERIODIC_INVEN_WW
0,201901,0.9
1,201902,0.8
2,201903,0.0
3,201904,0.0
4,201905,1.2
...,...,...
204,202248,0.0
205,202249,0.0
206,202250,0.0
207,202251,0.0
